numerical instabilities may occur when:
- very close to objects, specifically edges and corners

- close the z-axis in cylindrical symmetries

- at edge extensions

- at very large distances (>100 x object size)

In [2]:
import magpylib as mp

In [2]:
'''create magnet, create sensor, position, orientation'''
#create cubiod with polarisation 1T pointing in x-dir, sides 1,2,3cm
cube = mp.magnet.Cuboid(polarization=(1,0,0), dimension=(0.03,0.02,0.03))

#create a sensor for measuring the field
sensor = mp.Sensor()
sensor.move((-0.01,0,0))
sensor.rotate_from_angax(angle=45, axis='z')
print(f"sensor's position: {sensor.position}")

from scipy.spatial.transform import Rotation as R
cube.position = (0.01,0,0)
cube.orientation = R.from_rotvec((30,0,0), degrees=True)
print(f"cube's position: {cube.position}")

mp.show(cube, sensor)

sensor's position: [-0.01  0.    0.  ]
cube's position: [0.01 0.   0.  ]


In [3]:
'''field computation'''
points = [(0,0,0), (0.1,0.1,0.1)] #<-- faster than python loop
B = mp.getB(cube, points)
print("B:\n", B.round(2))
H = mp.getH(cube, points)
print("H:\n", H.round(2))

B:
 [[ 0.62  0.    0.  ]
 [-0.    0.    0.  ]]
H:
 [[-3.0534626e+05  0.0000000e+00  0.0000000e+00]
 [-3.4310000e+01  2.3142000e+02  2.3055000e+02]]


In [8]:
'''collection of objects'''
obj1 = mp.Sensor()
obj2 = mp.magnet.Cuboid(polarization=(1,0,0), dimension=(0.03,0.02,0.03))

collection = mp.Collection(obj1)
collection.add(obj2)

#you can now do stuff to the whole group, or interact with individual objects
collection.rotate_from_angax(angle=45, axis='z')

Collection(id=4517970000)

In [30]:
'''complex shapes'''
import numpy as np

points = np.array(
    [
        (-1, -1, 0),
        (-1, 1, 0),
        (1, -1, 0),
        (1, 1, 0),
        (0, 0, 2),
    ])

pyramid = mp.magnet.TriangularMesh.from_ConvexHull(magnetization = (0,1e6,1e6), points=points)
pyramid.show()

In [3]:
import numpy as np
positions = np.array([0,0,0],[1,1,1])
magnet = magpy.Cuboid(polarization=(1,0,0), dimension=(0.03,0.02,0.03), positions=positions)
magnet.show()

TypeError: Field elements must be 2- or 3-tuples, got '1'

In [3]:
magnet = mp.magnet.Cuboid(polarization = (0.715,0.746,0),
                             dimension = (1.61, 1.63,1),
                             position = (14.225, -12.9, 0)
                             )

H = mp.getH(magnet)

MagpylibBadUserInput: No observers provided
Observers must be either
- array_like positions of shape (N1, N2, ..., 3)
- Sensor object
- Collection with at least one Sensor
- 1D list of the above
received None of type 'NoneType' instead.

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


magnet = mp.magnet.Cuboid(polarization = (0.715,0.746,0),
                          dimension = (1.61, 1.63,1),
                          position = (14.225, -12.9, 0))

x = np.arange(-15 / 2,
                      15 / 2 + 0.1,
                      0.1)

y = np.arange(-15 / 2,
                      15 / 2 + 0.1,
                      0.1)

X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)

points = np.column_stack([X.ravel(), Y.ravel(), Z.ravel()])

H = mp.getH(magnet, points)

H = H[:, :2]

#reshape from ~90,000, 2 --> 301, 301, 2
H = tf.reshape(H, [301,
                        301,
                             2])

plt.imshow(H)
plt.show()

2025-12-18 09:10:00.594146: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2025-12-18 09:10:00.594166: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2025-12-18 09:10:00.594171: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2025-12-18 09:10:00.594198: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-12-18 09:10:00.594229: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


InvalidArgumentError: {{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:GPU:0}} Input to reshape is a tensor with 45602 values, but the requested shape has 181202 [Op:Reshape]